In [1]:
import os
import sys

from dotenv import load_dotenv
import pandas as pd
import requests

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [2]:
load_dotenv()

CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')
REFRESH_TOKEN = os.getenv('REFRESH_TOKEN')

In [9]:
# In your NOTEBOOK
import os

from dotenv import load_dotenv

load_dotenv()
print(f'NOTEBOOK IS USING: {os.environ.get("CLIENT_ID")}')  # Add this line

NOTEBOOK IS USING: 137549


In [10]:
from src.ingestion.auth.strava_auth import get_access_token
from src.ingestion.extractors.strava_exctractor import StravaEndpoints

activities_url = StravaEndpoints.get_activities()
access_token = get_access_token('cloud')

header = {'Authorization': f'Bearer {access_token}'}

cloud
137549 1184ee4b53c17a4caec26f1fe6b1f0169d1136a4 c7b0516c8801fde1965ea36e40428b81941ba448
Refreshing Strava access token...
Successfully refreshed access token.


In [11]:
access_token

'0569b7e048dfac808c1eb3e364fb09ba16556e83'

In [7]:
request_page_num = 1
all_activities: list[str] = []

while True:
    param = {'per_page': 200, 'page': request_page_num}
    my_dataset = requests.get(activities_url, headers=header, params=param).json()

    if len(my_dataset) == 0:
        break

    if all_activities:
        all_activities.extend(my_dataset)

    else:
        all_activities = my_dataset

    request_page_num += 1

In [ ]:
from src.ingestion.extractors.strava_exctractor import StravaExtractor

extractor = StravaExtractor(access_token)
all_activities_new = extractor.fetch_all_activities()
all_activities_new

In [ ]:
len(all_activities)

In [ ]:
df_all_activities = pd.json_normalize(all_activities)
df_all_activities.head()

# Exploring columns

In [ ]:
df_all_activities.columns

In [ ]:
df_all_activities.dtypes

In [ ]:
with pd.option_context('display.max_columns', None):
    display(df_all_activities.head())

In [ ]:
df_all_activities.has_kudoed.unique()

## Filter out unimportant columns

In [ ]:
unimportant_columns = [
    'resource_state',
    'location_city',
    'location_state',
    'location_country',
    'commute',
    'flagged',
    'start_latlng',
    'end_latlng',
    'heartrate_opt_out',
    'external_id',
    'upload_id',
    'upload_id_str',
    'from_accepted_tag',
    'has_kudoed',
    'athlete_resource_state',
    'map_resource_state',
]

In [ ]:
df_trunc = df_all_activities.drop(columns=unimportant_columns, errors='ignore')
df_trunc